Notebook used to resolve the deltas to the appropriate users/comments. For each set of comments we can obtain a list of the comments and users who received a delta.

In [38]:
import pickle
posts = pickle.load(open('post_info.p','rb'))
comments = pickle.load(open('comment_info.p','rb'))

Get the post and comments for a single thread. We can use this as an example to write the code.

In [39]:
post_ids = list(posts.keys())

In [40]:
p = post_ids[1]

In [41]:
pst = posts[p]
cmt = comments[p]

In [42]:
pst

{'author': 'TheRedMacbook',
 'name': 't3_5c8u5v',
 'num_comments': 45,
 'score': 0,
 'text': 'In this past election, millions of eligible Americans did not come out to vote.  Millions more in the primaries as well.  This interactive from the NYT shows what percentage of eligible voters actually went out to nominate Trump and Clinton.  \n\nhttp://www.nytimes.com/interactive/2016/08/01/us/elections/nine-percent-of-america-selected-trump-and-clinton.html\n\nIn the recent referendum to vote "Leave" for the UK, many Remain supporters stated they didn\'t bother coming out because they didn\'t think that Leave would actually win.  All around me I see a slightly similar phenomenon with the 2016 Presidential Election.  Had more individuals come out to the primaries, neither Trump nor Clinton would have been nominated.  \n_____\n\n> *Hello, users of CMV! This is a footnote from your moderators. We\'d just like to remind you of a couple of things. Firstly, please remember to* ***[read through our

In [43]:
op = pst['author']
pname = pst['name']
print("OP: ", op, " Post name: ", pname, " Post ID: ", p)

OP:  TheRedMacbook  Post name:  t3_5c8u5v  Post ID:  5c8u5v


In [44]:
for c in cmt:
    if c['delta']:
        print(c['author'])

TheRedMacbook
DeltaBot
TheRedMacbook
DeltaBot


We see that there are four comments with a delta. Two are by the OP and two are by the DeltaBot. We can resolve the delta using either.

Let's look at each post in more detail:

In [45]:
for c in cmt:
    if c['delta']:
        print(c['author'])
        print(c['text'])
        print('\n')

TheRedMacbook
∆ I guess I never really considered the lack of education that is rampant among non-voters.  I still believe though that voter turnout is too low, even for two candidates that people did not like very much.  


DeltaBot
Confirmed: 1 delta awarded to /u/______NSA______ ([1∆](/r/changemyview/wiki/user/______NSA______)).

^[Delta System Explained](https://www.reddit.com/r/changemyview/wiki/deltasystem) ^| ^[Deltaboards](https://www.reddit.com/r/changemyview/wiki/deltaboards)
[​](HTTP://DB3PARAMSSTART
{
  "comment": "This is hidden text for DB3 to parse. Please contact the author of DB3 if you see this",
  "issues": {},
  "parentUserName": "______NSA______"
}
DB3PARAMSEND)


TheRedMacbook
∆ I didn't really think about that to be 100% honest.  Perhaps at the polls they could implement two crash course paragraphs for each candidate next to each of their ballots with a list of their policies on all major and some minor issues?


DeltaBot
Confirmed: 1 delta awarded to /u/jt4 ([13

It looks like the deltabot is awarding the delta to the parent user who wrote the comment OP is responding to. Let's see if we can recover this directly.

To do so we need to find the comments with the ID that corresponds to the parent ID. There is an additional string processing step necessary to do this.

In [49]:
for c in cmt:
    if c['delta']:
        parent = c['parent'].split('_')[1]
        for k in cmt:
            if k['id'] == parent:
                print(k['author'])

______NSA______
TheRedMacbook
jt4
TheRedMacbook


We see that the parent users are those who were awarded a delta by the deltabot. The parents of the deltabot comments are the OP. We can correctly resolve the deltas by ignoring the deltabot.

In [52]:
for c in cmt:
    if c['author'] != "DeltaBot":
        if c['delta']:
            parent = c['parent'].split('_')[1]
            for k in cmt:
                if k['id'] == parent:
                    print(k['author'])

______NSA______
jt4


We can now package this into a function.

In [59]:
def resolve_deltas(comments):
    """This function takes a set of comments in response to a post and 
    returns the usernames and comment IDs of the users who the OP 
    awarded a delta
    
    Returns
    
    list of tuples : Each tuple is an author name, comment ID pair."""
    #TODO: This could be made more efficient by turning comments into a dict rather
    # than a list of dicts.
    D = []
    for c in comments:
        if c['author'] != "DeltaBot": # Ignore deltabot
            if c['delta']:
                parent = c['parent'].split('_')[1]
                for k in comments: # loop through again to find parent
                    if k['id'] == parent:
                        D.append((k['author'],k['id']))
    return D

In [60]:
resolve_deltas(cmt)

[('______NSA______', 'd9uiyap'), ('jt4', 'd9uiy5n')]

Now we can try to run this for the entire dataset:

In [61]:
deltas_dict = {}
for p in post_ids:
    # Get comments for that post
    deltas = resolve_deltas(comments[p])
    deltas_dict[p] = deltas

# Store deltas object in pickle
pickle.dump(deltas_dict, open('deltas_info.p','wb'))

In [62]:
deltas_dict

{'5c8xdc': [('Ansuz07', 'd9ujw71')],
 '5c8u5v': [('______NSA______', 'd9uiyap'), ('jt4', 'd9uiy5n')],
 '5c8txh': [('Salanmander', 'd9ul5l1')],
 '5c8rgf': [('Ansuz07', 'd9ujuku')],
 '5c8pmk': [('Ansuz07', 'd9uj8pk')],
 '5c8ph5': [],
 '5c8ozp': [],
 '5c8lj7': [],
 '5c8dgo': [],
 '5c8cfv': [],
 '5c89vs': [],
 '5c7ynz': [],
 '5c7xfm': [('whitepalms85', 'd9uelu3')],
 '5c7xcy': [],
 '5c7ky1': [],
 '5c7jc8': [('Hans_Brickface', 'd9urlc0')],
 '5c7a1i': [],
 '5c6u3x': [],
 '5c6tkn': [],
 '5c6m45': [],
 '5c660h': [],
 '5c60ep': [('InsufficientOverkill', 'd9u8tay')],
 '5c5wg9': [],
 '5c5onw': [('DHCKris', 'd9twc9g'),
  ('StellaAthena', 'd9v08nj'),
  ('Lmsaylor', 'd9v2x6c')],
 '5c5eza': [],
 '5c597h': [('bguy74', 'd9tv8em')],
 '5c57s6': [],
 '5c55cg': [('Generic_On_Reddit', 'd9twgsz')],
 '5c4os1': [],
 '5c4nx8': [],
 '5c4kri': [('Panda413', 'd9tlrc0')],
 '5c4hsf': [],
 '5c4h1a': [],
 '5c4evu': [],
 '5c4cnj': [],
 '5c45vg': [],
 '5c45pv': [('AnythingApplied', 'd9tkipl')],
 '5c43xi': [],
 '5c3zl0': 